## Apr 2021 kaggle TPS

This notebook carries out initial exploration of the training data to get an understanding of the distribution and completeness of each column and relationship to the target. 

The aim is to build a broad understanding, enough to make some small feature engineering decisions - and make some baseline predictions.

Synthanic [Data Dictionary](https://www.kaggle.com/c/tabular-playground-series-apr-2021/data?select=train.csv)

In [ ]:
# imports for EDA
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.mosaicplot import mosaic

In [ ]:
# imports for inference and scoring
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score, log_loss, make_scorer
from xgboost import XGBClassifier

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
sns.set_style("whitegrid")
sns.set_context("paper")

### EDA
We can get some idea of what to expect from the first few rows - next to look at the distribution of each more closely, and think about how to handle missing data and how to encode sex, class, port of embarkation etc.

In [ ]:
df = pd.read_csv("../data/raw/train.csv")
df.head()

In [ ]:
df.info()

Note null values in `Age`, `Ticket`, `Fare`, `Cabin` and `Embarked`

In [ ]:
df.describe()

So with a 43% survival rate, hopefully we can do better than predicting everyone as lost at 57%. Noticing a lot of missing ages, not so many missing fares. 

For the purposes of this challend I'm assuming the passenger IDs are all ok - so on to survival, and lots of plots.

In [ ]:
sns.countplot(data=df, x="Survived")
plt.show()

In [ ]:
sns.countplot(data=df, x="Pclass", hue="Survived")
plt.show()

In [ ]:
sns.countplot(data=df, x="Sex", hue="Survived")
plt.show()

No missing values here - but note significant differences in outcomes between male and female.

In [ ]:
df.groupby(['Sex', 'Survived'])['Survived'].count()

In [ ]:
mosaic(df, ['Pclass', 'Sex', 'Survived'])
plt.show()

In [ ]:
sns.displot(data=df, x="Age", hue="Survived", kind="kde")
plt.show()

Worth noting that the other seaborn displots `hist` and `ecdf` can show up some interesting quirks in the data

In [ ]:
sns.countplot(data=df, x="SibSp", hue="Survived")
plt.show()

The SibSp chart is dominated by the number of passengers with 0 or 1 - let's zoom in on 2+

In [ ]:
dplot = sns.countplot(data=df, x="SibSp", hue="Survived")
dplot.set(xlim=(1.5, 6.5))
dplot.set(ylim=(0.0, 2100.0))
plt.show()

In [ ]:
sns.countplot(data=df, x="Parch", hue="Survived")
plt.show()

In [ ]:
dplot = sns.countplot(data=df, x="Parch", hue="Survived")
dplot.set(xlim=(0.5, 7.5))
dplot.set(ylim=(0, 8000))
plt.show()

Values (and the lack of) from 5+ in `SibSp` and 4+ in `Parch` may lead to overfitting in the unseen data - binning these into broader categories could be more robust

In [ ]:
print("Number of unique tickets: {}".format(df['Ticket'].nunique()))
print("Number of null tickets: {}".format(df['Ticket'].isnull().sum()))

In [ ]:
sns.displot(data=df, x="Fare", hue="Survived", kind="kde")

Those paying < 100 did not do so well. Let's take a closer look at passengers who paid more:

In [ ]:
dplot = sns.displot(data=df, x="Fare", hue="Survived", kind="kde")
dplot.set(xlim=(100, 800))
dplot.set(ylim=(0, 0.001))
plt.show()

In [ ]:
print("Number of passengers missing embarkation port: {}".format(df['Embarked'].isnull().sum()))

In [ ]:
emb_fill = {'Embarked': 'X'}
df.fillna(value=emb_fill, inplace=True)

In [ ]:
df.groupby(['Embarked', 'Survived'])['Survived'].count()

In [ ]:
sns.countplot(data=df, x="Embarked", hue="Survived")
plt.show()

In [ ]:
df['Cabin'].head(10)

In [ ]:
print("Number of unique cabin IDs: {}".format(df['Cabin'].nunique()))
print("Number of null cabin IDs: {}".format(df['Cabin'].isnull().sum()))

In [ ]:
df['CabinArea'] = df['Cabin'].fillna('X').map(lambda x: x[0].strip())

In [ ]:
sns.countplot(data=df, x="CabinArea", hue="Survived")
plt.show()

In [ ]:
df['Ticket'].head(10)

In [ ]:
print("Number of unique ticket IDs: {}".format(df['Ticket'].nunique()))
print("Number of null ticket IDs: {}".format(df['Ticket'].isnull().sum()))

In [ ]:
df['TicketStub'] = df['Ticket'].fillna('X').map(lambda x:str(x).split()[0] if len(str(x).split()) > 1 else 'X') \
                               .str.upper() \
                               .replace("[^a-zA-Z0-9]*", "", regex=True)

In [ ]:
df['TicketStub'].unique()

Prior to cleaning, lots of similar tickets - for example 'SC/PARIS', 'S.C./PARIS', 'SC/Paris'. Decided to make all tickets uppercase and remove any punctuation.

Unsure if overkill but groups similar-ish tickets together and will process any tickets in the test set in exactly the same way. Some tickets are much more unlucky than others:

In [ ]:
df.groupby(['TicketStub'], sort=False, dropna=True)['Survived'].count()

In [ ]:
df.groupby(['TicketStub'], sort=False, dropna=True)['Survived'].mean()

### Modelling

After initial EDA, the aim is to put together some straightforward models:
- Baseline, assuming no survivors
- Logistic regression classifier
- Gradient boosting classifier

There is also work to be done to work out the best way to impute missing data around `Age`, `SibSp`, `Parch`, `Ticket`, `Cabin`, and `Embarked`. 

In [ ]:
target = df[['Survived']]
features = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'CabinArea', 'TicketStub']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

At this point I don't want to make any assumptions about the data, apart from separating categorical and numeric columns.

In [ ]:
categorical = ['Pclass', 'Sex', 'Embarked', 'CabinArea', 'TicketStub']
numeric = ['Age', 'SibSp', 'Parch', 'Fare']

In [ ]:
X_train[numeric].describe()

In [ ]:
sns.displot(data=X_train, x='Fare', kind='kde') 
plt.show()

The idea here is to replace missing values of `Fare` with the median of `Pclass`, and then retain that value for the test set:

In [ ]:
fare_map = X_train[['Fare', 'Pclass']].dropna().groupby('Pclass').median().to_dict()
X_train['Fare'] = X_train['Fare'].fillna(X_train['Pclass'].map(fare_map['Fare']))

In [ ]:
fare_map

In [ ]:
sns.displot(data=X_train, x='Age', kind='kde')
plt.show()

And to do something similar with `Age`

In [ ]:
age_map = X_train[['Age', 'Sex']].dropna().groupby(['Sex']).median().to_dict()
X_train['Age'] = X_train['Age'].fillna(X_train['Sex'].map(age_map['Age']))

In [ ]:
age_map

In [ ]:
sns.displot(data=X_train, x='Age', kind='kde')

In [ ]:
X_train.info()

With nulls filled and a train/test split specified, numeric columns are scaled with a `StandardScaler` and categorical columns are transformed with a `OneHotEncoder`.

In [ ]:
categorical = ['Pclass', 'Sex', 'Embarked', 'CabinArea', 'TicketStub']
numeric = ['Age', 'SibSp', 'Parch', 'Fare']

In [ ]:
preprocessor = make_column_transformer(
    (StandardScaler(), numeric),
    (OneHotEncoder(drop='if_binary'), categorical), 
    remainder='drop')

Initial estimator is `LogisticRegression` as a baseline for a binary classification task.

In [ ]:
model = make_pipeline(
    preprocessor,
    LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5))

_ = model.fit(X_train, np.ravel(y_train))

To set a floor for model performance, let's see a classification report as if the ship is predicted as going down with no survivors:

In [ ]:
y_pred = model.predict(X_train)
y_dummy = np.full(shape=len(y_pred), fill_value=0)
print(classification_report(y_train, y_dummy))

In [ ]:
print(classification_report(y_train, y_pred))

Performance on training data is ok at 77% overall accuracy; better than guessing that no one makes it at least, at 57% accuracy. Compare to the held out test set:

In [ ]:
X_test.info()

In [ ]:
# using age and fare maps from X_train, no peeking
X_test['Fare'] = X_test['Fare'].fillna(X_test['Pclass'].map(fare_map['Fare']))
X_test['Age'] = X_test['Age'].fillna(X_test['Sex'].map(age_map['Age']))

In [ ]:
y_unseen = model.predict(X_test)
print(classification_report(y_test, y_unseen))

To finish with the same steps are carried out on the competition test set and submitted. With more time you could do EDA on the test set to look for drift across each category, but for right now I'm interested in how this model performs without any additional intervention.

This initial submission achieved a leaderboard score of 0.77000. 

In [ ]:
model_gb = make_pipeline(
    preprocessor,
    XGBClassifier(objective="binary:logistic", random_state=42))

_ = model_gb.fit(X_train, np.ravel(y_train))

In [ ]:
y_pred_gb = model_gb.predict(X_train)
print(classification_report(y_train, y_pred_gb))

In [ ]:
y_unseen_gb = model_gb.predict(X_test)
print(classification_report(y_test, y_unseen_gb))

In [ ]:
model_gb

The ensemble method achieved a leaderboard score of 0.78642 out of the box. Let's see what a moderate parameter search yields. 

Using `mini_param` to test the grid search works as expected before kicking off thousands of runs with different classifiers

In [ ]:
clf = XGBClassifier() # SGDClassifier(), GradientBoostingClassifier()
scores = {'acc': make_scorer(accuracy_score), 
          'f1': make_scorer(f1_score)}

In [ ]:
pre_process = ColumnTransformer([
    ('num', StandardScaler(), numeric),
    ('cat', OneHotEncoder(), categorical)], n_jobs=-1)

In [ ]:
grid = GridSearchCV(clf, param_grid=xgb_param, refit=True, scoring='accuracy', cv=3, n_jobs=-1)

In [ ]:
X_prep = pre_process.fit_transform(X_train)
grid.fit(X_prep, np.ravel(y_train))

In [ ]:
grid.best_params_

In [ ]:
X_test_prep = pre_process.transform(X_test)
y_test_acc = grid.predict(X_test_prep)
print(classification_report(y_test, y_test_acc))

In [ ]:
grid_out = pd.DataFrame.from_dict(grid.cv_results_)

In [ ]:
grid_out_loc = "../data/scores/grid_out_xgb.csv"
grid_out.to_csv(grid_out_loc)

In [ ]:
pd.set_option("display.max_columns", None)
#grid_out.sort_values(['rank_test_f1','rank_test_acc'], ascending=[True, True]).head(5)
grid_out

In [ ]:
grid_out.sort_values(['rank_test_acc','rank_test_f1'], ascending=[True, True]).head(5)

Taking the parameters of the two `GradientBoostingClassifier` models with best f1 and accuracy for submission to kaggle

In [ ]:
acc_param = {'alpha': 0.001, 'class_weight': None, 'early_stopping': False, 'eta0': 0.01, 'fit_intercept': False, 'learning_rate': 'optimal', 'loss': 'modified_huber', 'max_iter': 1000, 'n_iter_no_change': 5, 'n_jobs': -1, 'penalty': 'l1', 'random_state': 42, 'shuffle': True, 'tol': 0.001, 'validation_fraction': 0.1, 'verbose': 1, 'warm_start': False}
f1_param = {'alpha': 0.001, 'class_weight': 'balanced', 'early_stopping': True, 'eta0': 0.01, 'fit_intercept': True, 'learning_rate': 'optimal', 'loss': 'modified_huber', 'max_iter': 1000, 'n_iter_no_change': 10, 'n_jobs': -1, 'penalty': 'l2', 'random_state': 42, 'shuffle': True, 'tol': 0.001, 'validation_fraction': 0.1, 'verbose': 1, 'warm_start': False}

In [ ]:
clf_acc = SGDClassifier(**acc_param).fit(X_prep, np.ravel(y_train))

In [ ]:
y_pred_acc = clf_acc.predict(X_prep)
print(classification_report(y_train, y_pred_acc))

In [ ]:
X_test_prep = pre_process.transform(X_test)
y_test_acc = clf_acc.predict(X_test_prep)
print(classification_report(y_test, y_test_acc))

In [ ]:
clf_f1 = SGDClassifier(**f1_param).fit(X_prep, np.ravel(y_train))

In [ ]:
y_pred_f1 = clf_f1.predict(X_prep)
print(classification_report(y_train, y_pred_f1))

In [ ]:
y_test_f1 = clf_f1.predict(X_test_prep)
print(classification_report(y_test, y_test_f1))

Both of these models seem consistent with each other - I don't think much of an improvement can be expected on the held out kaggle set.

In [ ]:
file_out = "../data/inference/cv_acc_xgb.csv"

In [ ]:
test_df.info()

Applying same transformations used on training set to test set:

In [ ]:
na_fill = {'Embarked': 'X', 'Cabin': 'X', 'Ticket': 'X'}
test_df.fillna(value=na_fill, inplace=True)
test_df['TicketStub'] = test_df['Ticket'].fillna('X').map(lambda x:str(x).split()[0] if len(str(x).split()) > 1 else 'X') \
                               .str.upper() \
                               .replace("[^a-zA-Z0-9]*", "", regex=True)
test_df['CabinArea'] = test_df['Cabin'].fillna('X').map(lambda x: x[0].strip())
test_df['Fare'] = test_df['Fare'].fillna(test_df['Pclass'].map(fare_map['Fare']))
test_df['Age'] = test_df['Age'].fillna(test_df['Sex'].map(age_map['Age']))
test_features = test_df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'CabinArea', 'TicketStub']]

In [ ]:
test_features['TicketStub'].unique()

In [ ]:
test_prep = pre_process.transform(test_features)

In [ ]:
test_xgb = grid.predict(test_prep)
test_xgb

In [ ]:
test_df['Survived'] = test_xgb.tolist()
test_submission = test_df[['PassengerId', 'Survived']]
test_submission.to_csv(file_out, index=False)

Although they differ in 1638 cases, both f1 and accuracy GBM models score 0.80394 on the leaderboard - perhaps they differ on the 75% held back for evaluation after the challenge ends.

In any case, cracking 80% accuracy on the public leaderboard caused a jump of hundreds of places, so it was good to see some positive impact from a good old fashioned grid search with a gradient boosting classifier.

Next step should be to look into feature importance identify the cases that are consistently predicted incorrectly.